# Intro
123

### TOC
[123](123)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import tensorflow as tf

tf.random.stateless_uniform(
    [5],
    (42,42),
    minval=0,
    maxval=10,
)


In [ ]:
for _ in range(5):
  print(tf.random.stateless_uniform(
    [10],
    (42,42),
    minval=0,
    maxval=10,
)
)

In [ ]:
import plotly.express as px
px.imshow([[0,1,0],[1,1,1],[0,1,1]])

In [ ]:
px.imshow(tf.image.stateless_random_flip_left_right(
  image=[[[0,211,0]],[[1,121,211]],[[0,211,1]]],
  # lower=0.1,
  # upper=0.9,
  # max_delta=0.02,
  seed=(42,42),
))

In [ ]:
tf.image.random_flip_left_right(
  image=[[[0,211,1]],[[1,121,211]],[[5,211,1]]],
  # lower=0.1,
  # upper=0.9,
  # max_delta=0.02,
  seed=42,
)